# Analyze city challenges to tax-exempt properties

In [1]:
from sqlalchemy import create_engine
import geopandas as gpd
import pandas as pd

In [2]:
db = create_engine('postgresql://jon@localhost:5432/propertydb')

In [3]:
parcels = pd.read_csv('input/pghexempt.csv')

### Load parcel data

In [4]:
parcelpoints = gpd.read_postgis("""SELECT parcelmbl, taxdesc, localtotal::int, ST_MakePoint(lon::float, lat::float) AS geom
FROM parcelcentroids
JOIN assessments
ON assessments.parcelid = parcelcentroids.parcelid
WHERE parcelmbl IN %(parcels)s;""", db, params={'parcels': tuple(parcels.parcelmbl)}, crs='EPSG:4269')
parcelpoints = parcelpoints.to_crs('EPSG:4326')

In [5]:
parcels = parcelpoints.merge(parcels, on='parcelmbl')

### Set attributes, export for Datawrapper

In [6]:
parcels['title'] = ''
parcels['tooltip'] = '<b>Parcel ID:</b> ' + parcels.parcelmbl + '<br><b>Owner:</b> ' + parcels.owner

parcels['color'] = 'black'
parcels.loc[parcels.owner == 'Allegheny General Hospital', 'color'] = '#008000'
parcels.loc[parcels.owner == 'Carnegie Mellon University', 'color'] = '#FF0000'
parcels.loc[parcels.owner == 'Presbyterian University Health System (UPMC)', 'color'] = '#800080'
parcels.loc[parcels.owner == 'University of Pittsburgh', 'color'] = '#0000FF'

In [7]:
parcels.to_file('output/pghexempt.geojson', driver='GeoJSON')

### Check outcome of challenges

In [8]:
pd.read_sql_query("""SELECT DISTINCT(asofdate) FROM assessments;""", db)

,asofdate
0,01-JUL-23


##### By tax status, taxable value

In [9]:
parcels.groupby('taxdesc').agg({ 'parcelmbl': 'count', 'localtotal': 'sum' })

,parcelmbl,localtotal
taxdesc,,
10 - Exempt,20,69862600
20 - Taxable,6,2749300


##### By owner

In [10]:
pd.pivot_table(parcels, index=['owner', 'taxdesc'], values='parcelmbl', aggfunc='count')

parcelmbl
owner                                        taxdesc                
8033 Bennet Street RE LLC                    20 - Taxable          1
Allegheny General Hospital                   10 - Exempt           1
Carnegie Mellon University                   10 - Exempt           1
                                             20 - Taxable          1
Community Options Inc.                       10 - Exempt           1
Karpeles Manuscript Library                  10 - Exempt           1
Mr. Property LLC                             20 - Taxable          1
NCSC USA Housing Development Corp.           10 - Exempt           1
Presbyterian University Health System (UPMC) 10 - Exempt           6
Private citizen                              10 - Exempt           7
                                             20 - Taxable          2
School Facility Development                  10 - Exempt           1
University of Pittsburgh                     20 - Taxable          1
Village of Shadyside Community Association   10 - Exempt           1